In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path='Models/lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [4]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [7]:
cap = cv2.VideoCapture('Videos/squat3.mp4')
stage = None
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0),192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Calculate angles
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    try:
        shoulder = [shaped[5][1], shaped[5][0]]
        hip = [shaped[11][1], shaped[11][0]]
        knee = [shaped[13][1], shaped[13][0]]
        ankle = [shaped[15][1], shaped[15][0]]

        
        angle = calculate_angle(shoulder, hip, knee)
        cv2.putText(frame, str(angle),(int(hip[0]), int(hip[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        angle2 = calculate_angle(hip, knee, ankle)
        cv2.putText(frame, str(angle2),(int(knee[0]), int(knee[1])), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        #squat logic
        if ((angle > 47) & (angle < 65)) | ((angle2 > 30) & (angle2 < 45)):
            stage = 'Good Squat'
        elif angle < 47:
            stage = 'Over Squat'
        elif (angle > 65) & (angle < 100):
            stage = 'Squat more'
        else:
            stage = 'Not squatted'
            
    except:
        pass
    
    cv2.rectangle(frame, (0,0), (440,73), (245,117,16), -1)
    # Stage data
    cv2.putText(frame, 'STAGE', (65,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, stage, 
                (60,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Thunder', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [8]:
keypoints_with_scores.shape

(1, 1, 17, 3)

In [ ]:
[nose - 0, 
 left eye - 1, 
 right eye - 2, 
 left ear - 3, 
 right ear - 4, d
 left shoulder - 5, 
 right shoulder - 6, 
 left elbow - 7, 
 right elbow - 8, 
 left wrist - 9, 
 right wrist - 10, 
 left hip - 11, 
 right hip - 12, 
 left knee - 13, 
 right knee - 14, 
 left ankle - 15, 
 right ankle - 16]